In [1]:
import os


In [2]:
%pwd

'f:\\Personal Project\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Personal Project\\Kidney-Disease-Classification-Deep-Learning-Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


#From this can access these are the variables from other file.Consider as an entity. Frozen means cannot change variable values
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [31]:
from cnn_classifier.constants import CONFIG_FILE_PATH
from src.cnn_classifier.utils.common import read_yaml,create_directories

# print(CONFIG_FILE_PATH)

ModuleNotFoundError: No module named 'cnn_classifier'

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


NameError: name 'CONFIG_FILE_PATH' is not defined